In [455]:
stage_button = widgets.Button(description = "Check data")
stage_button.on_click(check)

# Prepare and check data
This step takes around 3 minutes. 

In [456]:
stage_button

Ready to upload 16,197 cells!


# Save data
Once the data's been checked for correctness (all files are present, and none of the data has already been uploaded), save it!

In [458]:
save_button = widgets.Button(description = "Save data",background_color='Green',color = 'white')
save_button.on_click(add_new_data)
save_button

Just saved data!


# Delete past uploads
If you've made any mistake, and need to delete something you've uploaded, this is the place to do it. Just select the upload that you'd like to remove, and click `delete`.

In [459]:
widgets.HBox(children = [delete_button,delete_dropdown])